In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Churn_Modelling.csv")

In [2]:
data.drop(["RowNumber","CustomerId","Surname"],axis=1,inplace=True)

In [3]:
from sklearn.preprocessing import LabelEncoder
Le= LabelEncoder()
data["Gender"]= Le.fit_transform(data["Gender"])


In [4]:
#One Hot Encoding
Geo = pd.get_dummies(data["Geography"],drop_first=True)

In [5]:
data = pd.concat([data,Geo],axis=1)
X= data.iloc[:,0:9]
X = pd.concat([X,Geo],axis=1)
X.drop("Geography",axis=1,inplace=True)
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Germany,Spain
0,619,0,42,2,0.00,1,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,0,1
2,502,0,42,8,159660.80,3,1,0,0,0
3,699,0,39,1,0.00,2,0,0,0,0
4,850,0,43,2,125510.82,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,0,0
9996,516,1,35,10,57369.61,1,1,1,0,0
9997,709,0,36,7,0.00,1,0,1,0,0
9998,772,1,42,3,75075.31,2,1,0,1,0


In [6]:
Y = data.iloc[:,10]

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization
#from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
from keras.activations import relu,sigmoid

In [13]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [20]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=2)


layers = [(20,), (45, 30, 15), (40, 20)] 
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

grid_result = grid.fit(X_train, y_train)

#[grid_result.best_score_,grid_result.best_params_]



Epoch 1/30
50/50 - 0s - loss: 1.1290 - accuracy: 0.2333
Epoch 2/30
50/50 - 0s - loss: 0.8556 - accuracy: 0.3516
Epoch 3/30
50/50 - 0s - loss: 0.6838 - accuracy: 0.5705
Epoch 4/30
50/50 - 0s - loss: 0.5833 - accuracy: 0.7081
Epoch 5/30
50/50 - 0s - loss: 0.5314 - accuracy: 0.7617
Epoch 6/30
50/50 - 0s - loss: 0.4999 - accuracy: 0.7841
Epoch 7/30
50/50 - 0s - loss: 0.4863 - accuracy: 0.7922
Epoch 8/30
50/50 - 0s - loss: 0.4686 - accuracy: 0.8019
Epoch 9/30
50/50 - 0s - loss: 0.4677 - accuracy: 0.7975
Epoch 10/30
50/50 - 0s - loss: 0.4634 - accuracy: 0.8020
Epoch 11/30
50/50 - 0s - loss: 0.4553 - accuracy: 0.8017
Epoch 12/30
50/50 - 0s - loss: 0.4558 - accuracy: 0.8053
Epoch 13/30
50/50 - 0s - loss: 0.4507 - accuracy: 0.8016
Epoch 14/30
50/50 - 0s - loss: 0.4528 - accuracy: 0.8019
Epoch 15/30
50/50 - 0s - loss: 0.4446 - accuracy: 0.8028
Epoch 16/30
50/50 - 0s - loss: 0.4511 - accuracy: 0.8039
Epoch 17/30
50/50 - 0s - loss: 0.4468 - accuracy: 0.8067
Epoch 18/30
50/50 - 0s - loss: 0.4445 - 

In [21]:
print(grid_result.best_score_,grid_result.best_params_)

0.8572500109672546 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (45, 30, 15)}
